In [1]:
# Run this cell once to install required packages (may take a minute)
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost folium shap pyarrow

# Imports
import warnings
warnings.filterwarnings("ignore")
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve
import folium
from folium.plugins import HeatMap
import shap
plt.style.use('seaborn-v0_8') # Updated style name
%matplotlib inline

In [ ]:
# Configuration: change filename if needed
!unzip accidents_joined.zip
FNAME = "accidents_joined.csv"
# replace if your file name differs

# Load
df = pd.read_csv(FNAME, parse_dates=['timestamp'], infer_datetime_format=True)

# Basic required columns check
required = {'timestamp','lat','lon','is_accident'}
if not required.issubset(set(df.columns)):
    raise ValueError(f"CSV must contain columns: {required}. Found: {set(df.columns)}")

# Basic preprocessing & feature creation
df = df.sort_values('timestamp').reset_index(drop=True)
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek  # 0=Mon
df['month'] = df['timestamp'].dt.month
df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)

# cyclical encoding for time features
df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)
df['dow_sin'] = np.sin(2*np.pi*df['dayofweek']/7)
df['dow_cos'] = np.cos(2*np.pi*df['dayofweek']/7)

# spatial cell id: simple rounded lat/lon (approx grid)
df['cell_lat'] = df['lat'].round(3)
df['cell_lon'] = df['lon'].round(3)
df['cell_id'] = df['cell_lat'].astype(str) + "_" + df['cell_lon'].astype(str)

# create a basic historic accident rate per cell (all time)
cell_counts = df.groupby('cell_id')['is_accident'].sum().rename('cell_accidents')
cell_totals = df.groupby('cell_id').size().rename('cell_total')
cell_stats = pd.concat([cell_counts, cell_totals], axis=1)
cell_stats['cell_rate'] = cell_stats['cell_accidents'] / cell_stats['cell_total']
df = df.merge(cell_stats[['cell_rate']], left_on='cell_id', right_index=True, how='left')

# If weather columns exist, ensure numeric
for c in ['precip','temp','visibility','wind_speed']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# Fill small NaNs if any
df.fillna({'precip':0,'temp':df.get('temp', df['lat']).median() if 'temp' in df.columns else 0}, inplace=True)

Archive:  accidents_joined.zip
replace accidents_joined.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
plt.figure(figsize=(10,5))
hour_counts = df.groupby('hour')['is_accident'].sum()
sns.lineplot(x=hour_counts.index, y=hour_counts.values, marker='o')
plt.title('Accidents by Hour of Day')
plt.xlabel('Hour (0-23)')
plt.ylabel('Number of Accidents')
plt.xticks(range(0,24))
plt.grid(alpha=0.3)
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
dow_counts = df.groupby('dayofweek')['is_accident'].sum()
sns.barplot(x=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], y=dow_counts.values, palette='viridis')
plt.title('Accidents by Day of Week')
plt.ylabel('Number of Accidents')
plt.xlabel('')
plt.show()


In [ ]:
if 'precip' in df.columns:
    # bin precip into categories
    df['rain_flag'] = (df['precip'] > 0).astype(int)
    grp = df.groupby('rain_flag')['is_accident'].sum()
    plt.figure(figsize=(6,4))
    sns.barplot(x=['No Rain','Rain'], y=grp.values, palette=['#4c72b0','#dd8452'])
    plt.title('Accidents: Rain vs No Rain')
    plt.ylabel('Number of Accidents')
    plt.show()
else:
    print("Skipping weather plot: no 'precip' column found.")


In [ ]:
numcols = ['hour','dayofweek','month','is_weekend','cell_rate']
numcols += [c for c in ['precip','temp','visibility','wind_speed'] if c in df.columns]
numcols += ['lat','lon']  # include spatial coords if desired
corr = df[numcols + ['is_accident']].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Feature Correlation Heatmap')
plt.show()


In [ ]:
# center map
center = [df['lat'].mean(), df['lon'].mean()]
m = folium.Map(location=center, zoom_start=12, tiles='cartodbpositron')

# prepare heat data as [lat, lon, weight]
heat_data = df[['lat','lon','is_accident']].values.tolist()
HeatMap([[r[0], r[1], r[2]] for r in heat_data], radius=10, blur=12, max_zoom=13).add_to(m)

# save and display
m.save('accident_hotspots.html')
print("Saved Folium heatmap to accident_hotspots.html (open in browser)")
m  # in Jupyter this will render the map inline


In [ ]:
# Choose features for modeling
feature_cols = ['hour_sin','hour_cos','dow_sin','dow_cos','is_weekend','cell_rate']
for c in ['precip','temp','visibility','wind_speed']:
    if c in df.columns:
        feature_cols.append(c)
# optionally include lat/lon (careful about overfitting)
feature_cols += ['lat','lon']

# ensure no NaNs
X = df[feature_cols].fillna(0)
y = df['is_accident'].astype(int)

# train-test split (stratify to maintain class imbalance)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train/Test sizes:", X_train.shape, X_test.shape)


In [ ]:
# RandomForest
rf = RandomForestClassifier(n_estimators=300, class_weight='balanced_subsample', random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
proba_rf = rf.predict_proba(X_test)[:,1]

# XGBoost
xgb = XGBClassifier(n_estimators=500, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
# compute scale_pos_weight for imbalance
scale_pos_weight = (y_train==0).sum() / (y_train==1).sum() if (y_train==1).sum()>0 else 1
xgb.set_params(scale_pos_weight=scale_pos_weight)
xgb.fit(X_train, y_train)
proba_xgb = xgb.predict_proba(X_test)[:,1]

# Stacking (meta learner = logistic regression)
estimators = [('rf', rf), ('xgb', xgb)]
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=1000), n_jobs=-1, passthrough=False)
stack.fit(X_train, y_train)
proba_stack = stack.predict_proba(X_test)[:,1]


In [ ]:
fpr_rf, tpr_rf, _ = roc_curve(y_test, proba_rf)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, proba_xgb)
fpr_stack, tpr_stack, _ = roc_curve(y_test, proba_stack)

plt.figure(figsize=(8,6))
plt.plot(fpr_rf, tpr_rf, label=f'RF AUC={auc(fpr_rf,tpr_rf):.3f}')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGB AUC={auc(fpr_xgb,tpr_xgb):.3f}')
plt.plot(fpr_stack, tpr_stack, label=f'Stack AUC={auc(fpr_stack,tpr_stack):.3f}')
plt.plot([0,1],[0,1],'k--', alpha=0.4)
plt.title('ROC Curves')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
prec_rf, recall_rf, _ = precision_recall_curve(y_test, proba_rf)
prec_xgb, recall_xgb, _ = precision_recall_curve(y_test, proba_xgb)
prec_stack, recall_stack, _ = precision_recall_curve(y_test, proba_stack)

plt.figure(figsize=(8,6))
plt.plot(recall_rf, prec_rf, label=f'RF AP={average_precision_score(y_test, proba_rf):.3f}')
plt.plot(recall_xgb, prec_xgb, label=f'XGB AP={average_precision_score(y_test, proba_xgb):.3f}')
plt.plot(recall_stack, prec_stack, label=f'Stack AP={average_precision_score(y_test, proba_stack):.3f}')
plt.title('Precision-Recall Curves')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# choose stack; threshold 0.5
th = 0.5
y_pred_stack = (proba_stack >= th).astype(int)
cm = confusion_matrix(y_test, y_pred_stack, normalize='true')  # normalize by true labels
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt=".2f", cmap='Blues', xticklabels=['No','Yes'], yticklabels=['No','Yes'])
plt.title(f'Normalized Confusion Matrix (Stack, thresh={th})')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
plt.figure(figsize=(6,6))
prob_true, prob_pred = calibration_curve(y_test, proba_stack, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='Stack')
plt.plot([0,1],[0,1],'k--', alpha=0.4)
plt.title('Calibration Plot (Stacked Model)')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.legend()
plt.grid(alpha=0.3)
plt.show()
# Brier score
print("Brier score (stack):", brier_score_loss(y_test, proba_stack))


In [ ]:
# We'll use the xgboost model for SHAP TreeExplainer (already trained)
explainer = shap.TreeExplainer(xgb)
# Due to speed, sample subset for SHAP plots (e.g., 2000 rows or full if small)
sample_idx = np.random.choice(X_test.index, size=min(2000, len(X_test)), replace=False)
X_shap = X_test.loc[sample_idx]
shap_values = explainer.shap_values(X_shap)


In [ ]:
plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_shap, plot_type='bar', show=True)


In [ ]:
feat = 'precip' if 'precip' in X.columns else 'cell_rate'
plt.figure(figsize=(8,6))
shap.dependence_plot(feat, shap_values, X_shap, interaction_index=None, show=True)


In [ ]:
# pick a risky sample (highest predicted prob in test)
idx = X_test.index[np.argmax(proba_xgb)]  # sample index with highest XGB prob
X_sample = X_test.loc[[idx]]
sv_sample = explainer.shap_values(X_sample)

# Bar style local explanation
# sv = sv_sample[0] if isinstance(sv_sample, (list,tuple)) else sv_sample # Original line
sv = sv_sample[1] if isinstance(sv_sample, list) else sv_sample # Fixed line: select SHAP values for the positive class (index 1)
feature_imp = pd.Series(sv.flatten(), index=X_sample.columns).sort_values(ascending=False) # Flatten the array before creating Series
plt.figure(figsize=(8,4))
feature_imp.head(10).plot(kind='barh', color='tomato')
plt.title('Top positive SHAP contributions (sample)')
plt.gca().invert_yaxis()
plt.show()

# Optional: interactive force plot (not PNG)
try:
    shap.initjs()
    display(shap.force_plot(explainer.expected_value, sv_sample, X_sample))
except Exception as e:
    print("Interactive SHAP force plot not displayed in this environment.")

In [ ]:
# Save trained models (optional) using joblib
import joblib
joblib.dump(rf, 'rf_model.joblib')
joblib.dump(xgb, 'xgb_model.joblib')
joblib.dump(stack, 'stack_model.joblib')

# Save processed dataset sample for Tableau / report
df.to_csv('accidents_processed_sample.csv', index=False)
print("Saved models and processed dataset sample.")
